<a href="https://colab.research.google.com/github/yellomello/Georgian_DCC/blob/main/lec04_OrdersData_StreamingAnalytics_local.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Video Explanation: https://youtu.be/CT0DVOraCjY

In [51]:
!apt-get update #update linux
!apt-get install openjdk-8-jdk-headless -qq > /dev/null #download and install openjdk
!wget -q http://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop2.7.tgz #download spark binary (gunzip). -q: Turn off Wget’s output.
!tar xf spark-3.2.1-bin-hadoop2.7.tgz #extract the spark package
!pip install -q findspark #install the findspark package

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [3,282 kB]
Get:13 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1,359 kB]
Fetched 4,977 kB in 5s (931 

In [54]:
#set environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop2.7"

In [56]:
# From Spark 2.0, SparkSession provides a common entry point for a Spark application
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
spark.conf.set('spark.sql.shuffle.partitions', 6)
# spark.conf.set('spark.executor.memory', '2g')
spark

In [57]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
spark

In [58]:
%ls sample_data

anscombe.json*                mnist_test.csv         README.md*
california_housing_test.csv   mnist_train_small.csv
california_housing_train.csv  orders_data/


In [59]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [60]:
!unzip -q /content/drive/MyDrive/orders_data.zip -d sample_data/orders_data

replace sample_data/orders_data/orders_1.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [61]:
%ls sample_data/orders_data

orders_100.csv  orders_28.csv  orders_46.csv  orders_64.csv  orders_82.csv
orders_10.csv   orders_29.csv  orders_47.csv  orders_65.csv  orders_83.csv
orders_11.csv   orders_2.csv   orders_48.csv  orders_66.csv  orders_84.csv
orders_12.csv   orders_30.csv  orders_49.csv  orders_67.csv  orders_85.csv
orders_13.csv   orders_31.csv  orders_4.csv   orders_68.csv  orders_86.csv
orders_14.csv   orders_32.csv  orders_50.csv  orders_69.csv  orders_87.csv
orders_15.csv   orders_33.csv  orders_51.csv  orders_6.csv   orders_88.csv
orders_16.csv   orders_34.csv  orders_52.csv  orders_70.csv  orders_89.csv
orders_17.csv   orders_35.csv  orders_53.csv  orders_71.csv  orders_8.csv
orders_18.csv   orders_36.csv  orders_54.csv  orders_72.csv  orders_90.csv
orders_19.csv   orders_37.csv  orders_55.csv  orders_73.csv  orders_91.csv
orders_1.csv    orders_38.csv  orders_56.csv  orders_74.csv  orders_92.csv
orders_20.csv   orders_39.csv  orders_57.csv  orders_75.csv  orders_93.csv
orders_21.csv   orders_3.c

In [62]:
from pyspark.sql.types import *

file_location = "sample_data/orders_data/orders_*.csv"

# create schema
ordersSchema = StructType([
  StructField("Order_ID", DoubleType(), True),
  StructField("Country", StringType(), True),
  StructField("Province", StringType(), True),
  StructField("City", StringType(), True),
  StructField("Latitude", DoubleType(), True),
  StructField("Longitude", DoubleType(), True),
  StructField("TimeStamp", StringType(), True),
  StructField("Sales_Volume", DoubleType(), True)]
)

print(type(ordersSchema))

<class 'pyspark.sql.types.StructType'>


In [63]:
from pyspark.sql.functions import *

# create streaming DF
streamingDF = (
  spark
    .readStream
    .schema(ordersSchema)              # schema for the streaming DF
    .option("maxFilesPerTrigger", 1)  # Form a stream by triggering one file at a time
    .csv(file_location)
)

# Is it a streaming DF?
print(streamingDF.isStreaming)

True


In [64]:



# get total sales volume group by Province
streamingVolumeDF = (
  streamingDF
    .select("Province", "Sales_Volume")
    .groupBy("Province")
    .sum()
)


print(streamingVolumeDF.isStreaming)


True


In [74]:
query = (
  streamingVolumeDF
    .writeStream
    .format("memory")        # store in-memory table
    .queryName("orders_stream4")
    .outputMode("complete")
    .start()
)

In [78]:
spark.sql('''
 SELECT *
 FROM orders_stream4


 ''').show()



+--------+------------------+
|Province| sum(Sales_Volume)|
+--------+------------------+
|      MB|            798.81|
|      SK|           2204.96|
|      NB| 770.8499999999999|
|      AB|           35589.0|
|      NL|            126.84|
|      BC|2119.1000000000004|
|      NS| 965.5000000000001|
|      ON| 62384.29999999998|
|      QC|           7313.57|
+--------+------------------+

